In [2]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
from math import sqrt

In [9]:
def read(his_file,sim_num): 

# Middle of the vegetation patch  
    jmid=50; imid=45; 
# detect the presence of vegetation patch to know its extents (Currently hardwired
    imin_plant=41 ; imax_plant=50 ; jmin_plant=45 ; jmax_plant=54   
    
    nc = netCDF4.Dataset(his_file)
    mask = nc.variables['mask_rho'][:]

# read x,y
    x = nc.variables['x_rho'][:,:]
    y = nc.variables['y_rho'][:,:] 
    tot_time = nc.variables['ocean_time'] 
    
    imax=len(x[1,:])
    jmax=len(y[:,1])
    
    # read water depth
    depth = nc.variables['h'][:,:]
    theta_s = nc.variables['theta_s'][:]
    theta_b = nc.variables['theta_b'][:]
    depth_c = nc.variables['hc'][:]
    
#Get Stretching function 
    N=60 # Number of vertical levels, kgrid=1 MATLAB code copy 
    Np=N+1 
    ds=1.0/N
    s=np.zeros(Np)
    Csur=np.zeros(Np)
    for i in range(0,Np):
        lev=i
        s[i]=(lev-N)*ds
        Csur[i]=-s[i]**2
    
# Reshape 1D vertical variables to 3D so we can broadcast
    s.shape=(np.size(s),1,1)
    Csur.shape=(np.size(Csur),1,1)
    
# read 3D water level at all time steps
    eta = nc.variables['zeta'] 
    
# calculate the 4D field of z values (vertical coordinate)
    z0=(depth_c*s+depth*Csur)/(depth_c+depth)
    
# Extract the time frame for the last tidal cycle
    t_tidal=[]
    for t in range(1,len(tot_time)):
        if(eta[t,jmid,imid]<0.0 and eta[t+1,jmid,imid]>0.0):
            t_tidal.append(t)

    t_tidal_srt=t_tidal[-2]
    t_tidal_end=t_tidal[-1]
    print "Last tidal cycle starts and ends at this instance" 
    print t_tidal_srt,t_tidal_end   
    t_slice=t_tidal_end-t_tidal_srt+1
       
    bef_j=jmin_plant-5
    aft_j=jmax_plant+5
     
    ke_bef=0.0 
    ke_aft=0.0              
    ke=0.0 
    bot_ke=0.0 
    sur_ke=0.0
    # Calculate change in mean kinetic energy over last tidal cycle before and after the patch (3pts.)
    for t in range (t_tidal_srt,t_tidal_end+1): 
        u=nc.variables['u'][t,:,:,:]
        v=nc.variables['v'][t,:,:,:]
        w=nc.variables['w'][t,:,:,:]
        for j in range(jmin_plant-1,jmax_plant+1): 
            for i in range(imin_plant-1,imax_plant+1):
                
                u_c=0.5*(u[1,j,i]+u[1,j,i+1])
                v_c=0.5*(v[1,j+1,i]+v[1,j,i])
                bot_ke=sqrt(u_c*u_c+v_c*v_c)+bot_ke
    
                u_c=0.5*(u[N-1,j,i]+u[N-1,j,i+1])
                v_c=0.5*(v[N-1,j+1,i]+v[N-1,j,i])
                sur_ke=sqrt(u_c*u_c+v_c*v_c)+sur_ke
                
                ke_diff_topbot=sur_ke-bot_ke 
       
    mean_ke_diff_topbot=(ke_diff_topbot)/t_slice
    
    print "mean_ke_diff_topbot"
    print mean_ke_diff_topbot
    ke_diff_vals=[mean_ke_diff_topbot]
    return ke_diff_vals    

In [11]:
'''Output from vegetation module COAWST Mean KE within the vegetation patch [(Watt/m2 * m3)/time = Watt-mts/second] '''

total_sim_list=16

mean_ke_diff_topbot=[None]*total_sim_list  
pcnt_mean_ke_diff_topbot=[None]*total_sim_list
 
for sim_num in range(1,total_sim_list):
    path1='/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/'
    path2='veg_test_his_'+ str(sim_num)+'_ext' + '.nc'
    url=path1+path2
    print url 
    ke_diff_vals=read(url,sim_num) 
    mean_ke_diff_topbot[sim_num]=ke_diff_vals[0]

# 
min_mean_ke_diff_topbot=min(mean_ke_diff_topbot[1:total_sim_list])
#min_mean_bot_ke=min(mean_bot_ke[1:total_sim_list])
#min_mean_sur_ke=min(mean_sur_ke[1:total_sim_list])

fname='ke_diff_top_botout.txt'
f = open(fname, 'w')
f.write("Case_no, mean_ke_diff_topbot, pcnt_mean_ke_diff_topbot: \n")

for sim_num in range(1,total_sim_list):
    pcnt_mean_ke_diff_topbot[sim_num]=(mean_ke_diff_topbot[sim_num]-min_mean_ke_diff_topbot)* \
                                                             100.00/min_mean_ke_diff_topbot
    
    f.write("%s\t, %s\t, %s\t \n" % (sim_num, mean_ke_diff_topbot[sim_num], \
                                        pcnt_mean_ke_diff_topbot[sim_num])) 
      
                       
f.close()


/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_1_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_ke_diff_topbot
5.15609014089
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_2_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_ke_diff_topbot
4.16297335326
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_3_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_ke_diff_topbot
5.28328573868
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_4_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_ke_diff_topbot
5.30279271816
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_5_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_ke_diff_topbot
4.90301316362
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_6_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_ke_diff_t